In [8]:
import numpy as np
import pandas as pd

from src.constant import DATA_DIR, DATABASE_DIR
from src.database import DB
from src.database.queries import *
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_from_index_file
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver

In [9]:
generators = [
    "cluster_netgen",
    "compression",
    "expansion",
    "explosion",
    "grid",
    "cluster",
    "implosion",
    "linearprojection",
    "rotation",
    "uniform_portgen",
]

test_instances = TSP_from_index_file(filepath=DATA_DIR / "TSP" / "TEST" / "index.json")

id_to_generator_name = {
    instance.id(): instance.filepath.parts[-2] for instance in test_instances
}

id_to_name = {
    instance.id(): instance.filepath.parts[-1] for instance in test_instances
}

In [11]:
def agg_prefix(prefix):

    frames = []

    def agg_cost(x):
        return sum(x == TSP_LKH_Solver.MAX_COST)
        

    for idx, db_path in enumerate(DATABASE_DIR.glob(f"{prefix}-*.db")):
        db = DB(db_path)
        results = pd.read_sql_query("SELECT * FROM results", db._conn)
        series = (
            results.loc[results["prefix"].str.startswith("test")]
            .groupby(["instance_id", "prefix"])["cost"]
            .min()
            .reset_index()
            .groupby("instance_id")["cost"]
            .agg(agg_cost)
        )
        frames.append(series)
        

    df = (
        pd.concat(frames, axis=1)
        .sum(axis=1)
        .rename(prefix)
        .to_frame()
    )

    df["generator"] = df.index.map(id_to_generator_name)
    df["name"] = df.index.map(id_to_name)
    df = df.pivot_table(index="generator", columns="name", values=prefix).loc[generators, :]
    return df

n = 500
plain = agg_prefix(f"run-plain-{n}")
sur = agg_prefix(f"run-{n}-sur-50")

sur = sur.assign(generator="surrogate-50").set_index("generator", append=True)
plain = plain.assign(generator=f"plain-{n}").set_index("generator", append=True)

In [12]:
df = pd.concat([plain, sur], axis=0).sort_index(level=0).loc[generators]   
df["mean"] = df.mean(axis=1)

In [13]:
plain_total_mean = plain.mean(axis=1).mean(axis=0)
sur_total_mean = sur.mean(axis=1).mean(axis=0)
print(f"{plain_total_mean=:.2f}", f"{sur_total_mean=:.2f}")

plain_total_mean=0.74 sur_total_mean=0.56


In [14]:
plain_training = plain.iloc[:5].mean(axis=1).mean(axis=0)
sur_training = sur.iloc[:5].mean(axis=1).mean(axis=0)

plain_not_training = plain.iloc[5:].mean(axis=1).mean(axis=0)
sur_not_training = sur.iloc[5:].mean(axis=1).mean(axis=0)

print(f"{plain_training=:.2f}", f"{sur_training=:.2f}")
print(f"{plain_not_training=:.2f}", f"{sur_not_training=:.2f}")

plain_training=0.12 sur_training=0.00
plain_not_training=1.36 sur_not_training=1.12


In [15]:
# 
df.to_excel("tmp.xlsx")